<a href="https://colab.research.google.com/github/zaitunlah/bootcampITS/blob/master/Copy_of_Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Silakan Save a Copy dulu sebelum menggunakan notebook ini!***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Library yang akan digunakan

In [ ]:
#Import all relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 0. Metode untuk Transformasi Struktur Dataset

## Membuat DataFrame (Format List dengan Koma)

In [ ]:
dat = {'Trx': ['T1', 'T2', 'T3', 'T4', 'T5'],
        'Items': ['B,C,E', 'A,C,D', 'A,C,D,E', 'A,D', 'A,C,E']
        }

In [ ]:
df_sample = pd.DataFrame(dat)
df_sample

## Transformasi ke Format 1 Item per Baris

In [ ]:
df_sample['Items'].str.get_dummies(sep=',')

In [ ]:
df_sample = df_sample.join(df_sample['Items'].str.get_dummies(sep=','))
df_sample

#1. Persiapan Data#

## Upload Dataset

Upload file `data_transaksi_retail.xlsx`

(bisa langsung, atau melalui  Mount Drive)

##Membaca Data Set dan Eksplorasi Data##

In [ ]:
data=pd.read_excel('data_transaksi_retail.xlsx')

In [ ]:
data

In [ ]:
data.head()

In [ ]:
# Cek struktur dataset
data.info()

In [ ]:
# Cek apakah ada data kosong
data.isna().sum()

In [ ]:
# Ada berapa unique product?
print(data['Nama Barang'].nunique())
data['Nama Barang'].unique()

In [ ]:
# Apa produk yang paling sering dibeli?
data['Nama Barang'].value_counts()

In [ ]:
# buat visualisasi top 10 barang laris

df_laris = data['Nama Barang'].value_counts().reset_index()
df_laris.columns = ['Nama Barang', 'Transaksi']
sns.barplot(x='Nama Barang', y='Transaksi', data = df_laris.head(10))

In [ ]:
# Cek jumlah transaksi
df_summary = data.groupby('Kode Transaksi').agg({'Nama Barang': ['count'], 'Quantity': ['sum']}).reset_index()
df_summary

In [ ]:
df_summary.columns = ['Kode Transaksi','Jenis Barang', 'Jumlah Barang']
df_summary

In [ ]:
# Distribusi Jumlah Barang per Transaksi
sns.histplot(df_summary['Jumlah Barang'])

## Pre-processing

Kita perlu untuk mengubah formatnya agar tiap kolom merepresentasikan 1 item, dengan nilai 1 (ada) atau 0 (tidak ada)

In [ ]:
data_basket = (data.groupby(['Kode Transaksi', 'Nama Barang'])['Quantity'].count()\
                                      .unstack().reset_index().fillna(0)\
                                      .set_index('Kode Transaksi'))

In [ ]:
data_basket

In [ ]:
# Show a subset of columns
data_basket.iloc[:,[0,1,2,3,4,5,6,7]].head()

In [ ]:
# Melakukan proses encoding -> Mengubah data kebentuk angka, agar sistem atau komputer dapat memahami informasi dari dataset
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

data_sets = data_basket.applymap(encode_units)
data_sets.head(5)

Tujuan dalam encoding ini adalah memastikan jika barang kurang dari sama dengan 0 maka keranjang tersebut bernilai 0 dan jika lebih dari 1 maka nilainya adalah 1, sehingga jika sebuah nota membeli barang A sebanyak 10 buah maka hanya akan dihitung 1.

#2. Association Rules Mining

## Membuat Frequent Itemsets

In [ ]:
frequent_itemsets = apriori(data_sets, min_support=0.1, use_colnames=True)
frequent_itemsets

## Membuat Association Rules

In [ ]:
rules0_1 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rules0_1

## Eksplorasi Frequent Itemsets

In [ ]:
frequent_itemsets = apriori(data_sets, min_support=0.1, use_colnames=True)
frequent_itemsets

In [ ]:
# Menentukan size tiap itemset
frequent_itemsets['k'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

In [ ]:
# Mengurutkan dari yang support tertinggi
frequent_itemsets.sort_values('support', ascending=False)

In [ ]:
# Filter frequent itemset dengan item tertentu
frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: 'Celana Jogger Casual' in x)]

## Praktek

Dengan data `df_sample` di atas, coba cari semua frequent itemsets dengan kriteria:
- minimum support 0.3
- mengandung item Ceri
- Diurutkan dari support tertinggi

Ada berapa itemset yang ditemukan? Berapa nilai support terkecil yang terambil?

In [ ]:
# tulis code di sini



Dengan frequent itemset di atas, buatlah association rules dengan confidence >= 0.7

Ada berapa rules yang diperoleh? Berapa nilai lift tertinggi?

In [ ]:
# tulis code di sini



# Kesimpulan

Algoritma apriori adalah salah satu algoritma yang merupakan penerapan praktis dari Market Basket Analysis (MBA). Algoritma ini digunakan untuk menganalisa banyaknya kombinasi produk yang terjadi di dalam transaksi ritel, yang akan sulit dan lama jika dilakukan secara manual.

Secara teknis, algoritma apriori akan mencari tingkat asosiasi antar item di dalam banyak kombinasi kelompok data secara otomatis. Kombinasi ini juga bisa disusun dengan suatu aturan (rule) asosiasi "Jika membeli ini produk A maka akan membeli produk B", sehingga algoritma ini dikategorikan sebagai Association Rules di ranah machine learning.

Dengan menemukan paket produk yang asosiasinya kuat, Anda sebagai seorang data scientist dapat menyarankan kepada bisnis dapat melakukan berbagai action item seperti membuat paket produk dengan penawaran khusus, mendekatkan produk-produk tersebut saling berdekatan dalam satu rak, mengeluarkan rekomendasi produk di sistem e-commerce, mengurangi masalah stok, dan lain-lain.